# Exploratory Pipeline for the Cognitive Science and Artificial Intelligence thesis "The Key Motif"

Zenodo repository:
https://zenodo.org/records/11061100

Files needed:

- _"Locibase.json"_
- _"esm2_embeddings_rbp.csv"_
- _"phage_host_interactions.csv"_
- _"RBPbase.csv"_


Files generated:
- _"esm2_embeddings_loci_per_protein.csv"_ <br>
Contains the host protein embeddings for each locus protein
<br>
- _"all_interactions_no_embeddings.csv"_ <br>
Contains phage-host interactions, without ESM-2 embeddings (to make it lighter) <br>
- _"kaptive_results.tsv"_ <br>
Contains K-loci information for each host, extracted using Kaptive <br>
- _"protein_sequences_K#_positive.fasta"_ <br>
Contains the aminoacid sequences of the Receptor-Binding Proteins of all phages that infect hosts of the K# locus, concatenated per phage <br>


# Obtaining individual host proteins

generates "esm2_embeddings_loci_per_protein.csv" from "Locibase.json"

In [ ]:
!pip install fair-esm

In [ ]:
import json
import pandas as pd
import numpy as np
import torch
import esm
from tqdm import tqdm

def compute_esm2_embeddings_loci_per_protein(general_path, data_suffix='', add=False):
    """
    This function computes ESM-2 embeddings for each individual protein within loci, from the Locibase.json file.

    INPUTS:
    - general path to the project data folder
    - data suffix to optionally add to the saved file name (default='')
    OUTPUT: esm2_embeddings_loci_per_protein.csv (with one embedding per protein)
    """

    # Load ESM-2 model
    model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
    batch_converter = alphabet.get_batch_converter()
    model.eval()  # disables dropout for deterministic results

    # Load json file
    with open(general_path + '/Locibase' + data_suffix + '.json') as dict_file:
        loci_dict = json.load(dict_file)

    # if embeddings already exist, to append new ones to them
    if add:
        old_embeddings_df = pd.read_csv(general_path + '/esm2_embeddings_loci_per_protein' + data_suffix + '.csv')
        processed_accession_proteins = set(zip(old_embeddings_df['accession'], old_embeddings_df['protein_index']))
        for key in list(loci_dict.keys()):
            loci_dict[key] = [seq for i, seq in enumerate(loci_dict[key]) if (key, i) not in processed_accession_proteins]
        print('Processing', sum(len(v) for v in loci_dict.values()), 'more protein sequences (add=True)')

    # Compute embeddings per protein
    protein_representations = []
    accessions = []
    protein_indices = []

    for key in tqdm(loci_dict.keys(), desc="Embedding loci proteins"):
        for idx, sequence in enumerate(loci_dict[key]):
            data = [(f"{key}_prot_{idx}", sequence)]
            batch_labels, batch_strs, batch_tokens = batch_converter(data)
            with torch.no_grad():
                results = model(batch_tokens, repr_layers=[33], return_contacts=True)
            token_representations = results["representations"][33]
            protein_embedding = token_representations[0, 1 : len(sequence) + 1].mean(0).numpy()

            accessions.append(key)
            protein_indices.append(idx)
            protein_representations.append(protein_embedding)

    # Save results
    embeddings_df = pd.concat([
        pd.DataFrame({'accession': accessions, 'protein_index': protein_indices}),
        pd.DataFrame(protein_representations)
    ], axis=1)

    if add:
        embeddings_df = pd.concat([old_embeddings_df, embeddings_df], axis=0, ignore_index=True)

    embeddings_df.to_csv(general_path + '/esm2_embeddings_loci_per_protein' + data_suffix + '.csv', index=False)
    print("Saved embeddings to:", general_path + '/esm2_embeddings_loci_per_protein' + data_suffix + '.csv')

    return embeddings_df


loci_path = "" # "path_to_folder_containing_Locibase.json"

compute_esm2_embeddings_loci_per_protein(loci_path)

# Obtaining confirmed Infections-only dataset ('all_interactions_no_embeddings.csv')

generates 'all_interactions_no_embeddings.csv' from 'esm2_embeddings_loci_per_protein.csv', 'esm2_embeddings_rbp.csv' and 'phage_host_interactions.csv'

adds to it the protein sequences from "RBPbase.csv", to generate "all_infections.csv"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from xgboost import XGBClassifier
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, auc, roc_curve
import matplotlib.pyplot as plt
import pickle
import os.path



embeddings_loci_protein = pd.read_csv("esm2_embeddings_loci_per_protein.csv") # generated above
embeddings_rbp = pd.read_csv("esm2_embeddings_rbp.csv")
phage_host_interactions = pd.read_csv('phage_host_interactions.csv')

# Create a single dataset that has host, phage, and interactions, but not embeddings

interactions_melted = phage_host_interactions.melt(
    id_vars=['Unnamed: 0'], var_name='phage_ID', value_name='label'
).rename(columns={'Unnamed: 0': 'accession'})

interactions_melted = interactions_melted.dropna(subset=['label'])

merged = interactions_melted.merge(embeddings_loci_protein, on='accession', how='inner')
merged = merged.merge(embeddings_rbp, on='phage_ID', how='inner')

final_df = merged[['accession', 'phage_ID', 'protein_ID', "label"]]

print(len(final_df))
final_df.drop_duplicates(inplace=True)
final_df.reset_index(drop=True, inplace=True)
print(len(final_df))

final_df.to_csv('all_interactions_no_embeddings.csv', index=False)
print("✅ Final per-protein dataframe saved as 'all_interactions_no_embeddings.csv'.")

In [ ]:
# adds Receptor-Binding Proteins to the interactions (no embeddings) file
interactions_no_embeddings = pd.read_csv("all_interactions_no_embeddings.csv")
RBProteins = pd.read_csv("RBPbase.csv")

RBProteins = RBProteins[["protein_ID", "protein_sequence"]]
RBProteins.head()

proteins_no_embeddings = pd.merge(interactions_no_embeddings, RBProteins, how = "left", left_on = "protein_ID", right_on = "protein_ID")

proteins_no_embeddings.to_csv("all_infections.csv", index = False)

# Using Kaptive to determine K-loci

requires the download and unzipping of "klebsiella_genomes.zip"

generates "kaptive_results.tsv"

In [ ]:
!pip install kaptive
!apt-get install minimap2

In [ ]:
# K-LOCUS EXTRACTION:

!kaptive assembly kpsc_k /content/drive/MyDrive/PhageHostLearn_Data/Kaptive/fasta_files/*.fasta -o kaptive_results.tsv -j -p

# 8mins to run

# Downloading concatenated RBPs of phages that infect specific K-loci as fasta files

requires "all_infections.csv" and "kaptive_results.tsv"
generates a .fasta file that contains for each phage infecting a host that belongs to a certain K-locus its proteins, concatenated

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df_sero = pd.read_csv("kaptive_results.tsv", sep="\t")

df_sero.head(2)

,Assembly,Best match locus,Best match type,Match confidence,Problems,Identity,Coverage,Length discrepancy,Expected genes in locus,"Expected genes in locus, details",Missing expected genes,Other genes in locus,"Other genes in locus, details",Expected genes outside locus,"Expected genes outside locus, details",Other genes outside locus,"Other genes outside locus, details","Truncated genes, details","Extra genes, details"
0,102KP-HG,KL151,unknown (KL151),Typeable,NaN,99.67%,100.00%,-3 bp,17 / 21 (80.95%),"KL151_01_galF,99.33%,100.00%;KL151_02_cpsACP,9...",NaN,0,NaN,0 / 21 (0.00%),NaN,1,"KL150_19_gmd,90.72%,100.00%",NaN,NaN
1,1210,KL22,K22,Typeable,NaN,99.72%,100.00%,1 bp,14 / 18 (77.78%),"KL22_01_galF,100.00%,100.00%;KL22_02_cpsACP,99...",NaN,0,NaN,0 / 18 (0.00%),NaN,2,"KL40_05_rfaG,99.18%,96.58%;KL150_19_gmd,100.00...",NaN,NaN


In [ ]:
df_sero["Best match type"].nunique()
# 1280 + 87

87

In [ ]:
1280 + 87

1367

In [ ]:
import pandas as pd

all_infections = pd.read_csv("all_infections.csv")
df_sero = pd.read_csv("kaptive_results.tsv", sep="\t")

In [ ]:
all_infections.head(2)

,accession,phage_ID,protein_ID,label,protein_sequence
0,ERS739095,K1PH164C1,K1PH164C1_gp1,1.0,MAFSWQEQIKPAGTQDIQCDIEYLDKSYIHVYLDGAETTGYTWTSA...
1,ERS739095,K1PH164C1,K1PH164C1_gp5,1.0,MAQSLEGTIQSLLQGVSQQVPRERQPGQLGAQLNMLSDPVSGIRRR...


In [ ]:
# Combine the infections information with the K-loci information
# ("Best match type" refers to the K-locus serotype of the host with that "accession")
df_sero = df_sero[["Assembly", "Best match type", "Match confidence"]]

sero_phage = pd.merge(all_infections, df_sero, how = "left", left_on = "accession", right_on="Assembly").drop("Assembly", axis=1)

sero_phage = sero_phage[sero_phage["Match confidence"] == "Typeable"]

sero_phage = sero_phage[sero_phage["Best match type"] != "Capsule null"]


sero_phage.head(2)

NameError: name 'df_sero' is not defined

In [ ]:
sero_phage[sero_phage["label"] == 1]["Best match type"].value_counts()[:10]


for el in list(sero_phage[sero_phage["label"] == 1]["Best match type"].value_counts().index)[1:]:
  df_onehost_analyze = sero_phage[(sero_phage["Best match type"] == el) & (sero_phage["label"] == 1)]

  # Only show K-loci serotypes with 7 unique infecting phages
  if df_onehost_analyze.phage_ID.nunique() == 7:
    print(el)
    print("Number of Phage proteins: ", len(df_onehost_analyze))
    print("Number of Phages: ", df_onehost_analyze.phage_ID.nunique())
    print()

K36
Number of Phage proteins:  25
Number of Phages:  7

K11
Number of Phage proteins:  20
Number of Phages:  7

K60
Number of Phage proteins:  19
Number of Phages:  7



In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# pos = 1: Positive phages
# pos = 0: Negative phages

# Re-run for each K-locus for which enough phages have been identified to generate results
serot = "K13"

# Only check confirmed positive interactions, as even confirmed negatives could
# have failed for a number of reasons beyond the initial attachment
for pos in [1]:
  # Change the label check from 0 to 1 and rename the output to switch from positive to negative interactions
  df_onehost = sero_phage[(sero_phage["Best match type"] == serot) & (sero_phage["label"] == pos)]

  print("Number of Phage proteins: ", len(df_onehost))
  print("Number of Phages: ", df_onehost.phage_ID.nunique())

  # Only keep the first occurrence of each protein per phage
  df_onehost = df_onehost.drop_duplicates(subset=["phage_ID", "protein_sequence"], keep="first")

  print("Number of Unique Phage-Proteins couples: ", len(df_onehost))
  print("Number of Unique Proteins: ", df_onehost.protein_sequence.nunique())


  # Group by phage_ID and concatenate protein sequences
  df_grouped = df_onehost.groupby("phage_ID")["protein_sequence"].apply(lambda x: "".join(x)).reset_index()

  # Rename column for clarity
  df_grouped.rename(columns={"protein_sequence": "concatenated_sequence"}, inplace=True)

  continue

  # Specify the output FASTA file name
  if pos:
    output_fasta = "protein_sequences_" + str(serot) + "_positive.fasta"
  else:
    output_fasta = "protein_sequences_" + str(serot) + "_negative.fasta"

  # Convert DataFrame to SeqRecord format
  records = [SeqRecord(Seq(seq), id=protein_id, description="")
            for protein_id, seq in zip(df_grouped["phage_ID"], df_grouped["concatenated_sequence"])]

  # Write to FASTA file
  SeqIO.write(records, output_fasta, "fasta")

  print(f"✅ FASTA file saved as '{output_fasta}'")

Number of Phage proteins:  46
Number of Phages:  9
Number of Unique Phage-Proteins couples:  28
Number of Unique Proteins:  25


## Load the .fasta file generated above to MEME, specifying "One Occurrence Per Sequence" and "99" as Maximum Width (under Advanced Options).


https://meme-suite.org/meme/

Create a folder "MEME_results". Within, create a subfolder with the name of the K-locus being analyzed (es: K11).

Once a job has been run:
- Go to "MEME HTML output"
- Look for the motif with the lowest E-value
- Submit/Download -> Download Motif
- Download both "fasta" (downloads a .txt that has easily readable information about the position of the motif) and "Minimal MEME" (downloads a .meme has full output information)
- Save them into "MEME_results/K#/"


# Identify in which protein the motif identified resides

In [ ]:
import os
import re
import pandas as pd

folder_path = "MEME_results"
# Path to the "MEME_results" folder. Inside one folder for each K-locus.
# Inside .meme and .txt files downloaded from MEME for most common motif found

subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

print(subfolders)


['K1', 'K16', 'K13', 'K18', 'K12', 'K2', 'K24', 'K14', 'K11', 'K19', 'K29', 'K33', 'K32', 'K26', 'K39', 'K36', 'K30', 'K3', 'K31', 'K25', 'K54', 'K63', 'K57', 'K58', 'K60', 'K46', 'K61', 'K47', 'K43', 'K49', 'K8', 'K64', 'K68', 'K74', 'K65', '.ipynb_checkpoints']


In [ ]:
path = "MEME_results/" +  serot

extension = ".txt"

files = [f for f in os.listdir(path) if f.endswith(extension)]
print(files)

# Path to your file
file_path = os.path.join(path, files[0])

# Initialize a list to store extracted data
extracted_data = []

# Open and read the file
with open(file_path, "r") as file:
    lines = file.readlines()

# Process the file line by line
for i in range(len(lines)):
    line = lines[i].strip()

    # Check if the line starts with ">"
    if line.startswith(">"):
        # Extract the identifier before "_"
        identifier = line.split("_")[0][1:]  # Remove ">"

        # Extract the offset number using regex
        offset_match = re.search(r"offset=\s*(\d+)", line)
        offset = offset_match.group(1) if offset_match else None

        # Get the next line (sequence) and compute its length
        if i + 1 < len(lines):
            sequence = lines[i + 1].strip()
            seq_length = len(sequence)

        # Store extracted data
        extracted_data.append((identifier, offset, seq_length, sequence))

# Display results
all_extracted = pd.DataFrame(extracted_data, columns=["Identifier", "Offset", "Sequence Length", "motif"])

all_extracted.Offset = all_extracted.Offset.apply(lambda x: int(x))

# Information extracted from the fasta (.txt) files
all_extracted

['RPDNGAEFGQGNISEFHVTTIGVYNFRSDAQWYVKSNPPEIGNQWGPFWSESTRPLR_fasta.txt']


,Identifier,Offset,Sequence Length,motif
0,K30lambda2,2719,57,RPDNGAEFGQGNISEMHVTTVGVYNFRSDAQWYVKSNPPEIGNQWG...
1,K7PH164C4,2717,57,RPDNGAEFGQGNISEMHVTTVGVYNFRSDAQWYVKSNPPEIGNQWG...
2,K65PH164,3644,57,NGAYASLYFQEYVGNFHQAIINVNGFGRDDSFYFRAGGDFICTRNG...
3,K62PH164C2,460,57,NGGTGSTNASDARLAFGLRMIDVPSNTSGATRCIKIATIKSPGAAG...


In [ ]:
# Function to compute sequential protein identifier per phage_ID
def compute_sequential_identifier(df):
    df["sequential_protein_identifier"] = ""  # Initialize the column
    start_pos = {}  # Dictionary to track start positions for each phage_ID

    for index, row in df.iterrows():
        phage_id = row["phage_ID"]
        seq_length = len(row["protein_sequence"])

        # Get the starting position for this phage_ID
        start = start_pos.get(phage_id, 0)
        end = start + seq_length

        # Assign the range as a string
        df.at[index, "sequential_protein_identifier"] = f"{start}-{end}"

        # Update the start position for the next protein under the same phage_ID
        start_pos[phage_id] = end

    return df

df_onehost["lengths"] = df_onehost["protein_sequence"].apply(lambda x: len(x))

# Apply the function
df_onehost_length = compute_sequential_identifier(df_onehost)

# Display the first few rows
df_onehost_length.head()

,accession,phage_ID,protein_ID,label,protein_sequence,Best match type,Match confidence,lengths,sequential_protein_identifier
332,ERS706555,K2PH164C2,K2PH164C2_gp55,1.0,MTNIKARKGGSSSPRTPVEMPDNLISKDKVKLLLAVSDGEVVNDFS...,K13,Typeable,1234,0-1234
333,ERS706555,K2PH164C2,K2PH164C2_gp59,1.0,MALYRQGKAAMDANGIVTGTGTNWQSALTLIRPGATILFLSSPIQM...,K13,Typeable,763,1234-1997
456,ERS706555,K2alfa62,K2alfa62_gp51,1.0,MAADGTVTGTGTKWQSSLSLIRPGATIMFLSSPIQMAVVNKVVSDT...,K13,Typeable,897,0-897
457,ERS706555,K2alfa62,K2alfa62_gp53,1.0,MTNIKARKGGSSKPRTPVEMPDNLISKDKIKLLLAVSDGEVVNDFS...,K13,Typeable,1267,897-2164
737,ERS706555,K7PH164C4,K7PH164C4_gp52,1.0,MAITTRIIAQQVTALDGANSRVSKYPKFTVQLGYSVSSLAATELLD...,K13,Typeable,679,0-679


In [ ]:
# Initialize the new column as False
df_onehost_length["motif_found"] = False

# Iterate over extracted_df and find matching rows in df
for _, row in all_extracted.iterrows():
    identifier = row["Identifier"]
    offset = row["Offset"]
    seq_length = row["Sequence Length"]
    target_range = (offset, offset + seq_length)  # Range for motif

    # Filter rows where phage_ID matches Identifier
    matching_rows = df_onehost_length[df_onehost_length["phage_ID"] == identifier]

    # Iterate through matching rows to check range inclusion
    for index, match_row in matching_rows.iterrows():
        start, end = map(int, match_row["sequential_protein_identifier"].split("-"))

        # Check if the target range is within this row's sequential identifier range
        if start <= target_range[0] and end >= target_range[1]:
            df_onehost_length.at[index, "motif_found"] = True
            break  # Ensure only one row is marked True per Identifier

df_onehost_length

,accession,phage_ID,protein_ID,label,protein_sequence,Best match type,Match confidence,lengths,sequential_protein_identifier,motif_found
332,ERS706555,K2PH164C2,K2PH164C2_gp55,1.0,MTNIKARKGGSSSPRTPVEMPDNLISKDKVKLLLAVSDGEVVNDFS...,K13,Typeable,1234,0-1234,True
333,ERS706555,K2PH164C2,K2PH164C2_gp59,1.0,MALYRQGKAAMDANGIVTGTGTNWQSALTLIRPGATILFLSSPIQM...,K13,Typeable,763,1234-1997,False
456,ERS706555,K2alfa62,K2alfa62_gp51,1.0,MAADGTVTGTGTKWQSSLSLIRPGATIMFLSSPIQMAVVNKVVSDT...,K13,Typeable,897,0-897,False
457,ERS706555,K2alfa62,K2alfa62_gp53,1.0,MTNIKARKGGSSKPRTPVEMPDNLISKDKIKLLLAVSDGEVVNDFS...,K13,Typeable,1267,897-2164,True
737,ERS706555,K7PH164C4,K7PH164C4_gp52,1.0,MAITTRIIAQQVTALDGANSRVSKYPKFTVQLGYSVSSLAATELLD...,K13,Typeable,679,0-679,False
738,ERS706555,K7PH164C4,K7PH164C4_gp55,1.0,MRQILPSAKAYLANNDKIRLAYLVSIELPGSTGNNAVYAYMTDYMR...,K13,Typeable,948,679-1627,False
739,ERS706555,K7PH164C4,K7PH164C4_gp61,1.0,MLYSLMRESRVVIEYDGRAYGFDALSDYTAGTSYEEFKANRRTIHK...,K13,Typeable,295,1627-1922,False
740,ERS706555,K7PH164C4,K7PH164C4_gp62,1.0,MSVQLLRNTRIFVSTVTTGFSKANTQEILVQDDVSWSQDSNSTDIT...,K13,Typeable,375,1922-2297,False
741,ERS706555,K7PH164C4,K7PH164C4_gp73,1.0,MGFYAGRIGDKKVLSLTSGNNKDVNNHTNPGWDTIFHSDMPHVVVL...,K13,Typeable,658,2297-2955,True
1780,ERS706555,K13PH07C1L,K13PH07C1L_gp44,1.0,MALVKATFVKDVDGQPWRFSSVAKMKAFNYSCYLGSSVFLESWHEG...,K13,Typeable,575,0-575,True


In [ ]:
import copy

full_onehost_length = copy.deepcopy(df_onehost_length)

full_onehost_length

,accession,phage_ID,protein_ID,label,protein_sequence,Best match type,Match confidence,lengths,sequential_protein_identifier,motif_found
332,ERS706555,K2PH164C2,K2PH164C2_gp55,1.0,MTNIKARKGGSSSPRTPVEMPDNLISKDKVKLLLAVSDGEVVNDFS...,K13,Typeable,1234,0-1234,True
333,ERS706555,K2PH164C2,K2PH164C2_gp59,1.0,MALYRQGKAAMDANGIVTGTGTNWQSALTLIRPGATILFLSSPIQM...,K13,Typeable,763,1234-1997,False
456,ERS706555,K2alfa62,K2alfa62_gp51,1.0,MAADGTVTGTGTKWQSSLSLIRPGATIMFLSSPIQMAVVNKVVSDT...,K13,Typeable,897,0-897,False
457,ERS706555,K2alfa62,K2alfa62_gp53,1.0,MTNIKARKGGSSKPRTPVEMPDNLISKDKIKLLLAVSDGEVVNDFS...,K13,Typeable,1267,897-2164,True
737,ERS706555,K7PH164C4,K7PH164C4_gp52,1.0,MAITTRIIAQQVTALDGANSRVSKYPKFTVQLGYSVSSLAATELLD...,K13,Typeable,679,0-679,False
738,ERS706555,K7PH164C4,K7PH164C4_gp55,1.0,MRQILPSAKAYLANNDKIRLAYLVSIELPGSTGNNAVYAYMTDYMR...,K13,Typeable,948,679-1627,False
739,ERS706555,K7PH164C4,K7PH164C4_gp61,1.0,MLYSLMRESRVVIEYDGRAYGFDALSDYTAGTSYEEFKANRRTIHK...,K13,Typeable,295,1627-1922,False
740,ERS706555,K7PH164C4,K7PH164C4_gp62,1.0,MSVQLLRNTRIFVSTVTTGFSKANTQEILVQDDVSWSQDSNSTDIT...,K13,Typeable,375,1922-2297,False
741,ERS706555,K7PH164C4,K7PH164C4_gp73,1.0,MGFYAGRIGDKKVLSLTSGNNKDVNNHTNPGWDTIFHSDMPHVVVL...,K13,Typeable,658,2297-2955,True
1780,ERS706555,K13PH07C1L,K13PH07C1L_gp44,1.0,MALVKATFVKDVDGQPWRFSSVAKMKAFNYSCYLGSSVFLESWHEG...,K13,Typeable,575,0-575,True


In [ ]:
all_extracted.Offset = all_extracted.Offset.apply(lambda x: int(x))

In [ ]:
folder_path = "/content/drive/MyDrive/PhageHostLearn_Data/motifs_KO"

subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

print(subfolders)

# Function to compute sequential protein identifier per phage_ID
def compute_sequential_identifier(df):
    df["sequential_protein_identifier"] = ""  # Initialize the column
    start_pos = {}  # Dictionary to track start positions for each phage_ID

    for index, row in df.iterrows():
        phage_id = row["phage_ID"]
        seq_length = len(row["protein_sequence"])

        # Get the starting position for this phage_ID
        start = start_pos.get(phage_id, 0)
        end = start + seq_length

        # Assign the range as a string
        df.at[index, "sequential_protein_identifier"] = f"{start}-{end}"

        # Update the start position for the next protein under the same phage_ID
        start_pos[phage_id] = end

    return df


from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

first_iteration = True


for serot in [x for x in subfolders if x[0] == "K"]:

  ### Identify a K-loci group and take it as a subset

  print(serot)

  # Change the label check from 0 to 1 and rename the output to switch from positive to negative interactions
  df_onehost = sero_phage[(sero_phage["Best match type"] == serot) & (sero_phage["label"] == 1)]

  # Only keep the first occurrence of each protein per phage
  df_onehost = df_onehost.drop_duplicates(subset=["phage_ID", "protein_sequence"], keep="first")

  # Group by phage_ID and concatenate protein sequences
  df_grouped = df_onehost.groupby("phage_ID")["protein_sequence"].apply(lambda x: "".join(x)).reset_index()

  # Rename column for clarity
  df_grouped.rename(columns={"protein_sequence": "concatenated_sequence"}, inplace=True)


  ### Load the file where the motifs found are identified

  path = "/content/drive/MyDrive/PhageHostLearn_Data/motifs_KO/" +  serot

  extension = ".txt"

  files = [f for f in os.listdir(path) if f.endswith(extension)]

  # If no motifs found -> no .txt file -> next iteration
  if len(files) < 1:
    continue

  # Path to your file
  file_path = os.path.join(path, files[0])

  # Initialize a list to store extracted data
  extracted_data = []

  # Open and read the file
  with open(file_path, "r") as file:
      lines = file.readlines()

  # Process the file line by line
  for i in range(len(lines)):
      line = lines[i].strip()

      # Check if the line starts with ">"
      if line.startswith(">"):
          # Extract the identifier before "_"
          identifier = line.split("_")[0][1:]  # Remove ">"

          # Extract the offset number using regex
          offset_match = re.search(r"offset=\s*(\d+)", line)
          offset = offset_match.group(1) if offset_match else None

          # Get the next line (sequence) and compute its length
          if i + 1 < len(lines):
              sequence = lines[i + 1].strip()
              seq_length = len(sequence)

          # Store extracted data
          extracted_data.append((identifier, offset, seq_length, sequence))

  ### Dataset that shows where the motif was found within the concatenated proteins

  # Display results
  df_extracted = pd.DataFrame(extracted_data, columns=["Identifier", "Offset", "Sequence Length", "motif"])

  df_extracted.Offset = df_extracted.Offset.apply(lambda x: int(x))

  all_extracted = pd.concat([all_extracted, df_extracted], ignore_index=True)

  ### Dataset that keeps track of the start and end of each protein before they get concatenated

  df_onehost_length = compute_sequential_identifier(df_onehost)

  # Initialize the new column as False
  df_onehost_length["motif_found"] = False
  df_onehost_length["Offset"] = -1

  ### Add "motif_found" == True to the row of the protein that contains the motif

  # Iterate over extracted_df and find matching rows in df
  for _, row in df_extracted.iterrows():
      identifier = row["Identifier"]
      offset = row["Offset"]
      seq_length = row["Sequence Length"]
      target_range = (offset, offset + seq_length)  # Range for motif

      # Filter rows where phage_ID matches Identifier
      matching_rows = df_onehost_length[df_onehost_length["phage_ID"] == identifier]

      # Iterate through matching rows to check range inclusion
      for index, match_row in matching_rows.iterrows():
          start, end = map(int, match_row["sequential_protein_identifier"].split("-"))

          # Check if the target range is within this row's sequential identifier range
          if start <= target_range[0] and end >= target_range[1]:
              df_onehost_length.at[index, "motif_found"] = True
              df_onehost_length.at[index, "Offset"] = offset
              break  # Ensure only one row is marked True per Identifier

  if first_iteration:
    full_onehost_length = copy.deepcopy(df_onehost_length)
    first_iteration = False
  else:
    full_onehost_length = pd.concat([full_onehost_length, df_onehost_length], ignore_index=True)


full_onehost_length.drop_duplicates(inplace = True)

full_onehost_length.to_csv("full_onehost_length.csv", index=False)

['K1', 'K16', 'K13', 'K18', 'K12', 'K2', 'K24', 'K14', 'K11', 'K19', 'K29', 'K33', 'K32', 'K26', 'K39', 'K36', 'K30', 'K3', 'K31', 'K25', 'K54', 'K63', 'K57', 'K58', 'K60', 'K46', 'K61', 'K47', 'K43', 'K49', 'K8', 'K64', 'K68', 'K74', 'K65', '.ipynb_checkpoints']
K1
K16
K13
K18
K12
K2
K24
K14
K11
K19
K29
K33
K32
K26
K39
K36
K30
K3
K31
K25
K54
K63
K57
K58
K60
K46
K61
K47
K43
K49
K8
K64
K68
K74
K65


In [ ]:
all_extracted.to_csv("all_extracted_motifs.csv", index = True)

In [ ]:
full_onehost_length[full_onehost_length.motif_found]
# 143 examples of proteins with motifs found
# 28 K-loci

,accession,phage_ID,protein_ID,label,protein_sequence,Best match type,Match confidence,sequential_protein_identifier,motif_found,Offset
1,ERS739095,K1PH164C1,K1PH164C1_gp5,1.0,MAQSLEGTIQSLLQGVSQQVPRERQPGQLGAQLNMLSDPVSGIRRR...,K1,Typeable,318-1104,True,803
3,ERS739095,K8PH128,K8PH128_gp41,1.0,MALVSQSIKNLKGGISQQPEILRYPEQGSLQVNGWSSETEGLQKRP...,K1,Typeable,0-791,True,488
8,NTUH,A1c,A1c_gp57,1.0,MLSDPVSGLRRRPPAEIAWQSTIDNPGLDELFVEYVERGTDGRHLL...,K1,Typeable,969-1721,True,1420
9,NTUH,S9a,S9a_gp91,1.0,MVQKTGPNLGMNYGWDLGESGWKPGMDANMKKLDAVVNAAVLNIAN...,K1,Typeable,0-260,True,203
10,ERS706555,K2PH164C2,K2PH164C2_gp55,1.0,MTNIKARKGGSSSPRTPVEMPDNLISKDKVKLLLAVSDGEVVNDFS...,K13,Typeable,0-1234,True,1168
...,...,...,...,...,...,...,...,...,...,...
420,ERS659577,K74PH129C2,K74PH129C2_gp5,1.0,MALVSQSIKNLKGGISQQPEILRYPEQGSLQVNGWSSETEGLQKRP...,K74,Typeable,0-791,True,182
427,ERS702132,K7PH164C4,K7PH164C4_gp73,1.0,MGFYAGRIGDKKVLSLTSGNNKDVNNHTNPGWDTIFHSDMPHVVVL...,K65,Typeable,2297-2955,True,2717
432,ERS702132,K30lambda2,K30lambda2_gp175,1.0,MGFYAGRIGDKKVLSLTSGNNKDVNNHTNPGWDTIFHSDMPHVVVL...,K65,Typeable,2299-2957,True,2719
434,ERS702132,K62PH164C2,K62PH164C2_gp80,1.0,MATYKVGKVKINGNGLMTGTGTNWTAANALVRVGATVVLATNPVRI...,K65,Typeable,219-853,True,460


In [ ]:
full_onehost_found = full_onehost_length[full_onehost_length.motif_found]


# Function to calculate percentage_offset
def calculate_percentage_offset(row):
    try:
        # Extract start and end from sequential_protein_identifier
        start, end = map(int, row["sequential_protein_identifier"].split("-"))
        offset = row["Offset"]

        # Ensure the offset is within the range
        if start <= offset <= end:
            return ((offset - start) / (end - start)) * 100
    except:
        return None  # Handle conversion errors or missing values

    return None  # Default case if not computed

# Apply the function to compute percentage_offset
full_onehost_found["percentage_offset"] = full_onehost_found.apply(calculate_percentage_offset, axis=1)

full_onehost_found

<ipython-input-68-8432377b70ee>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_onehost_found["percentage_offset"] = full_onehost_found.apply(calculate_percentage_offset, axis=1)


,accession,phage_ID,protein_ID,label,protein_sequence,Best match type,Match confidence,sequential_protein_identifier,motif_found,Offset,percentage_offset
1,ERS739095,K1PH164C1,K1PH164C1_gp5,1.0,MAQSLEGTIQSLLQGVSQQVPRERQPGQLGAQLNMLSDPVSGIRRR...,K1,Typeable,318-1104,True,803,61.704835
3,ERS739095,K8PH128,K8PH128_gp41,1.0,MALVSQSIKNLKGGISQQPEILRYPEQGSLQVNGWSSETEGLQKRP...,K1,Typeable,0-791,True,488,61.694058
8,NTUH,A1c,A1c_gp57,1.0,MLSDPVSGLRRRPPAEIAWQSTIDNPGLDELFVEYVERGTDGRHLL...,K1,Typeable,969-1721,True,1420,59.973404
9,NTUH,S9a,S9a_gp91,1.0,MVQKTGPNLGMNYGWDLGESGWKPGMDANMKKLDAVVNAAVLNIAN...,K1,Typeable,0-260,True,203,78.076923
10,ERS706555,K2PH164C2,K2PH164C2_gp55,1.0,MTNIKARKGGSSSPRTPVEMPDNLISKDKVKLLLAVSDGEVVNDFS...,K13,Typeable,0-1234,True,1168,94.651540
...,...,...,...,...,...,...,...,...,...,...,...
420,ERS659577,K74PH129C2,K74PH129C2_gp5,1.0,MALVSQSIKNLKGGISQQPEILRYPEQGSLQVNGWSSETEGLQKRP...,K74,Typeable,0-791,True,182,23.008850
427,ERS702132,K7PH164C4,K7PH164C4_gp73,1.0,MGFYAGRIGDKKVLSLTSGNNKDVNNHTNPGWDTIFHSDMPHVVVL...,K65,Typeable,2297-2955,True,2717,63.829787
432,ERS702132,K30lambda2,K30lambda2_gp175,1.0,MGFYAGRIGDKKVLSLTSGNNKDVNNHTNPGWDTIFHSDMPHVVVL...,K65,Typeable,2299-2957,True,2719,63.829787
434,ERS702132,K62PH164C2,K62PH164C2_gp80,1.0,MATYKVGKVKINGNGLMTGTGTNWTAANALVRVGATVVLATNPVRI...,K65,Typeable,219-853,True,460,38.012618


In [ ]:
print(full_onehost_found.percentage_offset.mean())
print(full_onehost_found.percentage_offset.std())

51.7665458865937
28.495129571534097


In [ ]:
print("Mean motif length: ", all_extracted.motif.apply(lambda x: len(x)).mean())
print("motif std: ", all_extracted.motif.apply(lambda x: len(x)).std())

Mean motif length:  43.16755793226381
motif std:  20.42111344183336


In [ ]:
len(all_extracted)

561

In [ ]:
min(all_extracted.motif.apply(lambda x: len(x)))

21

In [ ]:
full_onehost_found["protein_length"] = full_onehost_found.protein_sequence.apply(lambda x: len(x))

full_onehost_found["protein_length"]

<ipython-input-84-0078319eb316>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_onehost_found["protein_length"] = full_onehost_found.protein_sequence.apply(lambda x: len(x))


,protein_length
1,786
3,791
8,752
9,260
10,1234
...,...
420,791
427,658
432,658
434,634


In [ ]:
full_onehost_found.to_csv("full_onehost_found.csv", index=False)

# AlphaFold Motif Search

In [ ]:
import pandas as pd


df = pd.read_csv("/content/full_onehost_found.csv")
motifs = pd.read_csv("/content/all_extracted_motifs.csv").drop("Unnamed: 0", axis=1)

df["Best match type"].value_counts()

In [ ]:
df[df["Best match type"] == "K29"].protein_ID.unique()

In [ ]:
prot = "K69PH164C2_gp22"
serot = "K29"

acc_K64 = df[df["Best match type"] == serot][df["protein_ID"] == prot]["phage_ID"].iloc[0]
offs_K64 = df[df["Best match type"] == serot][df["protein_ID"] == prot]["Offset"].iloc[0]
mot_len = motifs[motifs['Identifier'] == acc_K64][motifs["Offset"] == int(offs_K64)]["Sequence Length"].iloc[0]


print("Protein: ", prot)
print("phage_ID: ", acc_K64)
print("Offset: ", offs_K64)
print("Motif Length: ", mot_len)

In [ ]:
print(prot)

protein_seq = df[df["Best match type"] == serot][df["protein_ID"] == prot]["protein_sequence"].iloc[0]

print(protein_seq)
# Upload this protein in trimeric form to AlphaFold:
# https://alphafoldserver.com/

In [ ]:
protein_start = int(df[df["Best match type"] == serot][df["protein_ID"] == prot]["sequential_protein_identifier"].iloc[0].split("-")[0])
print("Motif Start: ", offs_K64 - protein_start)
print("Motif End: ", offs_K64 - protein_start + mot_len)
print(protein_seq[offs_K64 - protein_start:offs_K64 - protein_start + mot_len])
# Look for position Motif Start:Motif End in Mol Viewer
# https://molstar.org/viewer/

# Validating results: protein with motif found vs protein max score via tool

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd


df = pd.read_csv("/content/drive/MyDrive/PhageHostLearn_Data/full_onehost_found.csv")
motifs = pd.read_csv("/content/drive/MyDrive/PhageHostLearn_Data/all_extracted_motifs.csv").drop("Unnamed: 0", axis=1)
april_single = pd.read_csv("/content/drive/MyDrive/PhageHostLearn_Data/April_validation_predictions_all_groups.csv") # 1.00 grouping threshold
# april_single = pd.read_csv("/content/drive/MyDrive/PhageHostLearn_Data/validation_predictions_all_groups_75.csv") # 0.75 grouping threshold
# april_single = pd.read_csv("/content/drive/MyDrive/PhageHostLearn_Data/validation_predictions_motif_focus_100.csv") # trained only on one protein, 1.00 grouping threshold
# april_single = pd.read_csv("/content/drive/MyDrive/PhageHostLearn_Data/validation_predictions_motif_focus_80.csv") # trained only on one protein, .80 grouping threshold

# df["Best match type"].value_counts()

In [ ]:
len(df)

134

In [ ]:
motifs.head()

,Identifier,Offset,Sequence Length,motif
0,K30lambda2,2719,57,RPDNGAEFGQGNISEMHVTTVGVYNFRSDAQWYVKSNPPEIGNQWG...
1,K7PH164C4,2717,57,RPDNGAEFGQGNISEMHVTTVGVYNFRSDAQWYVKSNPPEIGNQWG...
2,K65PH164,3644,57,NGAYASLYFQEYVGNFHQAIINVNGFGRDDSFYFRAGGDFICTRNG...
3,K62PH164C2,460,57,NGGTGSTNASDARLAFGLRMIDVPSNTSGATRCIKIATIKSPGAAG...
4,A1c,1420,40,HLPRYVPGRVLQMQNSSVTNMAFMRFSGDRKSLLVYEFMW


In [ ]:
print(len(df)) # 134 phage-host pairs, with the motif found in protein_ID
df.head()

134


,accession,phage_ID,protein_ID,label,protein_sequence,Best match type,Match confidence,sequential_protein_identifier,motif_found,Offset,percentage_offset,protein_length
0,ERS739095,K1PH164C1,K1PH164C1_gp5,1.0,MAQSLEGTIQSLLQGVSQQVPRERQPGQLGAQLNMLSDPVSGIRRR...,K1,Typeable,318-1104,True,803,61.704835,786
1,ERS739095,K8PH128,K8PH128_gp41,1.0,MALVSQSIKNLKGGISQQPEILRYPEQGSLQVNGWSSETEGLQKRP...,K1,Typeable,0-791,True,488,61.694058,791
2,NTUH,A1c,A1c_gp57,1.0,MLSDPVSGLRRRPPAEIAWQSTIDNPGLDELFVEYVERGTDGRHLL...,K1,Typeable,969-1721,True,1420,59.973404,752
3,NTUH,S9a,S9a_gp91,1.0,MVQKTGPNLGMNYGWDLGESGWKPGMDANMKKLDAVVNAAVLNIAN...,K1,Typeable,0-260,True,203,78.076923,260
4,ERS706555,K2PH164C2,K2PH164C2_gp55,1.0,MTNIKARKGGSSSPRTPVEMPDNLISKDKVKLLLAVSDGEVVNDFS...,K13,Typeable,0-1234,True,1168,94.651540,1234


In [ ]:
april_single.head(3)
# 25'120 accession-phage-protein rows

# focus on successful interactions only (label=1.0)
# Add a score ranking grouped by accession-phage_ID, max score is #1
# Check how many times the protein with the motif is #1, how many times top-n
# Find way to normalize by number of phages within that phage (es: finding the top protein when only 1, not impressive.
# out of 8, impressive. Find way to check this relative position. 1/1 = 0, 1/8 = max, perhaps just number of proteins
# minus position over number of proteins, minus the average?)

,accession,phage_ID,protein_ID,label,score
0,KP_HGUA02_071,A1a,A1a_gp14,0.0,0.114446
1,KP_HGUA02_071,A1a,A1a_gp2,0.0,0.076832
2,KP_HGUA02_071,A1a,A1a_gp6,0.0,0.099417


In [ ]:
april_single['score_ranking'] = april_single.groupby(['accession', 'phage_ID'])['score'] \
                        .rank(method='dense', ascending=False) \
                        .astype(int)

successful_april = april_single[april_single.label == 1]
# successful_april = april_single[april_single.true_label == 1]

successful_april.head()

,accession,phage_ID,protein_ID,label,score,score_ranking
62,KP_HGUA02_071,A3d,A3d_gp40,1.0,0.056520,2
63,KP_HGUA02_071,A3d,A3d_gp45,1.0,0.114291,1
64,KP_HGUA02_071,A3d,A3d_gp46,1.0,0.031451,3
103,KP_HGUA02_071,S8b,S8b_gp2,1.0,0.793052,1
104,KP_HGUA02_071,S8c,S8c_gp1,1.0,0.860217,2


In [ ]:
# Step 1: Count number of unique protein_IDs per (accession, phage_ID) in the first dataframe
protein_counts = successful_april.groupby(['accession', 'phage_ID'])['protein_ID'].nunique()

# Step 2: Convert the index to a dictionary
protein_count_dict = protein_counts.to_dict()

# Step 3: Map the dictionary to a new column in the second dataframe
successful_april['protein_count'] = successful_april.apply(
    lambda row: protein_count_dict.get((row['accession'], row['phage_ID']), 0),
    axis=1
)


for i in range(2,9):
  # Group by accession and phage_ID, compute standard deviation of scores per group
  std_per_group = successful_april[successful_april["protein_count"] == i].groupby(['accession', 'phage_ID'])['score'].std()

  # Then take the average of all those standard deviations
  average_std = std_per_group.mean()

  print(i)
  print("Average standard deviation of scores per (accession, phage_ID):", average_std)
  print()

2
Average standard deviation of scores per (accession, phage_ID): 0.022723368905329003

3
Average standard deviation of scores per (accession, phage_ID): 0.025181270971947015

4
Average standard deviation of scores per (accession, phage_ID): 0.07823101481191316

5
Average standard deviation of scores per (accession, phage_ID): 0.02546013882495613

6
Average standard deviation of scores per (accession, phage_ID): 0.033656553504869166

7
Average standard deviation of scores per (accession, phage_ID): 0.008739460755227779

8
Average standard deviation of scores per (accession, phage_ID): 0.027948241418095703



<ipython-input-32-47615afbd25d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  successful_april['protein_count'] = successful_april.apply(


In [ ]:
successful_april.drop_duplicates(inplace=True)

<ipython-input-33-bc5f0ff156be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  successful_april.drop_duplicates(inplace=True)


In [ ]:
successful_april[successful_april["accession"] == "KP_HGUA02_071"][successful_april["phage_ID"] == "A3d"]

<ipython-input-34-d02e8ec4768a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  successful_april[successful_april["accession"] == "KP_HGUA02_071"][successful_april["phage_ID"] == "A3d"]


,accession,phage_ID,protein_ID,label,score,score_ranking,protein_count
62,KP_HGUA02_071,A3d,A3d_gp40,1.0,0.056520,2,3
63,KP_HGUA02_071,A3d,A3d_gp45,1.0,0.114291,1,3
64,KP_HGUA02_071,A3d,A3d_gp46,1.0,0.031451,3,3


In [ ]:
top_successful_april = successful_april[successful_april["score_ranking"] == 1]
# 333 phage-host couples
top_successful_april.head(2)

,accession,phage_ID,protein_ID,label,score,score_ranking,protein_count
63,KP_HGUA02_071,A3d,A3d_gp45,1.0,0.114291,1,3
103,KP_HGUA02_071,S8b,S8b_gp2,1.0,0.793052,1,1


In [ ]:
df.head(3)

,accession,phage_ID,protein_ID,label,protein_sequence,Best match type,Match confidence,sequential_protein_identifier,motif_found,Offset,percentage_offset,protein_length
0,ERS739095,K1PH164C1,K1PH164C1_gp5,1.0,MAQSLEGTIQSLLQGVSQQVPRERQPGQLGAQLNMLSDPVSGIRRR...,K1,Typeable,318-1104,True,803,61.704835,786
1,ERS739095,K8PH128,K8PH128_gp41,1.0,MALVSQSIKNLKGGISQQPEILRYPEQGSLQVNGWSSETEGLQKRP...,K1,Typeable,0-791,True,488,61.694058,791
2,NTUH,A1c,A1c_gp57,1.0,MLSDPVSGLRRRPPAEIAWQSTIDNPGLDELFVEYVERGTDGRHLL...,K1,Typeable,969-1721,True,1420,59.973404,752


In [ ]:
df

,accession,phage_ID,protein_ID,label,protein_sequence,Best match type,Match confidence,sequential_protein_identifier,motif_found,Offset,percentage_offset,protein_length
0,ERS739095,K1PH164C1,K1PH164C1_gp5,1.0,MAQSLEGTIQSLLQGVSQQVPRERQPGQLGAQLNMLSDPVSGIRRR...,K1,Typeable,318-1104,True,803,61.704835,786
1,ERS739095,K8PH128,K8PH128_gp41,1.0,MALVSQSIKNLKGGISQQPEILRYPEQGSLQVNGWSSETEGLQKRP...,K1,Typeable,0-791,True,488,61.694058,791
2,NTUH,A1c,A1c_gp57,1.0,MLSDPVSGLRRRPPAEIAWQSTIDNPGLDELFVEYVERGTDGRHLL...,K1,Typeable,969-1721,True,1420,59.973404,752
3,NTUH,S9a,S9a_gp91,1.0,MVQKTGPNLGMNYGWDLGESGWKPGMDANMKKLDAVVNAAVLNIAN...,K1,Typeable,0-260,True,203,78.076923,260
4,ERS706555,K2PH164C2,K2PH164C2_gp55,1.0,MTNIKARKGGSSSPRTPVEMPDNLISKDKVKLLLAVSDGEVVNDFS...,K13,Typeable,0-1234,True,1168,94.651540,1234
...,...,...,...,...,...,...,...,...,...,...,...,...
129,ERS659577,K74PH129C2,K74PH129C2_gp5,1.0,MALVSQSIKNLKGGISQQPEILRYPEQGSLQVNGWSSETEGLQKRP...,K74,Typeable,0-791,True,182,23.008850,791
130,ERS702132,K7PH164C4,K7PH164C4_gp73,1.0,MGFYAGRIGDKKVLSLTSGNNKDVNNHTNPGWDTIFHSDMPHVVVL...,K65,Typeable,2297-2955,True,2717,63.829787,658
131,ERS702132,K30lambda2,K30lambda2_gp175,1.0,MGFYAGRIGDKKVLSLTSGNNKDVNNHTNPGWDTIFHSDMPHVVVL...,K65,Typeable,2299-2957,True,2719,63.829787,658
132,ERS702132,K62PH164C2,K62PH164C2_gp80,1.0,MATYKVGKVKINGNGLMTGTGTNWTAANALVRVGATVVLATNPVRI...,K65,Typeable,219-853,True,460,38.012618,634


In [ ]:
# Merge the score_ranking column from successful_april into df2
df_scores = df.merge(
    successful_april[['accession', 'phage_ID', 'protein_ID', 'score_ranking']],
    on=['accession', 'phage_ID', 'protein_ID'],
    how='left'  # Keeps all rows in df, adds score_ranking where there's a match
)

In [ ]:
# Step 1: Count number of unique protein_IDs per (accession, phage_ID) in the first dataframe
protein_counts = successful_april.groupby(['accession', 'phage_ID'])['protein_ID'].nunique()

# Step 2: Convert the index to a dictionary
protein_count_dict = protein_counts.to_dict()

# Step 3: Map the dictionary to a new column in the second dataframe
df_scores['protein_count'] = df_scores.apply(
    lambda row: protein_count_dict.get((row['accession'], row['phage_ID']), 0),
    axis=1
)


In [ ]:
df_scores.protein_count.value_counts()

,count
protein_count,
5,42
2,37
3,28
1,18
6,5
4,3
8,1


In [ ]:
df_scores.score_ranking.value_counts()
# 18 cases with protein_count == 1

,count
score_ranking,
1,56
2,31
3,25
5,9
4,8
6,4
7,1


In [ ]:
df_scores[df_scores["protein_count"] == 5].score_ranking.value_counts()

,count
score_ranking,
3,13
5,9
1,9
4,8
2,3


In [ ]:
df_scores[df_scores["protein_count"] == 2].score_ranking.value_counts().loc[2]

np.int64(22)

In [ ]:
print("***** RESULTS AT 1.00 SIMILARITY GROUPING *****")
print()

group_sizes = []
num_phages = []
num_correct = []

for i in range(2, 9):
  if sum(df_scores[df_scores["protein_count"] == i].score_ranking.value_counts()) != 0:
    print("Number of proteins per phage: ", i)
    print("Total count: ", sum(df_scores[df_scores["protein_count"] == i].score_ranking.value_counts()))

    first_count = 0
    try:
      first_count = df_scores[df_scores["protein_count"] == i].score_ranking.value_counts().loc[1]
    except:
      pass
    group_sizes.append(i)
    num_phages.append(sum(df_scores[df_scores["protein_count"] == i].score_ranking.value_counts()))
    num_correct.append(first_count)
    print(num_correct)
    try:
      print("First position: ", round( first_count / (sum(df_scores[df_scores["protein_count"] == i].score_ranking.value_counts())), 3))
      print(first_count)
      print("Expected if Random: ", (1 / i))
    except ZeroDivisionError:
      print("DivisionByZero")
    print()

***** RESULTS AT 1.00 SIMILARITY GROUPING *****

Number of proteins per phage:  2
Total count:  37
[np.int64(15)]
First position:  0.405
15
Expected if Random:  0.5

Number of proteins per phage:  3
Total count:  28
[np.int64(15), np.int64(10)]
First position:  0.357
10
Expected if Random:  0.3333333333333333

Number of proteins per phage:  4
Total count:  3
[np.int64(15), np.int64(10), np.int64(3)]
First position:  1.0
3
Expected if Random:  0.25

Number of proteins per phage:  5
Total count:  42
[np.int64(15), np.int64(10), np.int64(3), np.int64(9)]
First position:  0.214
9
Expected if Random:  0.2

Number of proteins per phage:  6
Total count:  5
[np.int64(15), np.int64(10), np.int64(3), np.int64(9), np.int64(1)]
First position:  0.2
1
Expected if Random:  0.16666666666666666

Number of proteins per phage:  8
Total count:  1
[np.int64(15), np.int64(10), np.int64(3), np.int64(9), np.int64(1), 0]
First position:  0.0
0
Expected if Random:  0.125



In [ ]:
from scipy.stats import chisquare

# Example data (replace with your actual values)
# group_sizes = [2, 3, 4, 5]           # number of proteins per phage
# num_phages = [37, 28, 25, 30]        # number of phages in each group
# num_correct = [15, 10, 8, 7]         # correct identifications in each group

expected_raw = [k / n for k, n in zip(num_phages, group_sizes)]

# Normalize expected to match the total of observed
total_observed = sum(num_correct)
total_expected = sum(expected_raw)
scaling_factor = total_observed / total_expected
expected_scaled = [e * scaling_factor for e in expected_raw]

# Chi-squared test
chi2_stat, p_value = chisquare(f_obs=num_correct, f_exp=expected_scaled)

print(f"Chi-squared statistic: {chi2_stat:.4f}")
print(f"P-value: {p_value:.4f}")

Chi-squared statistic: 7.6491
P-value: 0.1767


In [ ]:
group_sizes

[2, 3, 4, 5, 6, 8]

In [ ]:
num_phages

[37, 28, 3, 42, 5, 1]

In [ ]:
num_correct

[np.int64(15), np.int64(10), np.int64(3), np.int64(9), np.int64(1), 0]

In [ ]:
from scipy.stats import binomtest

# Your data
group_sizes = [2, 3, 4, 5, 6, 8]
num_phages = [37, 28, 3, 42, 5, 1]
# num_correct = [32, 16, 1, 24, 4, 0]
num_correct = [15, 10, 3, 9, 1, 0]

# Run binomial tests
for n, k, x in zip(group_sizes, num_phages, num_correct):
    p_null = 1 / n  # random guessing probability
    result = binomtest(k=x, n=k, p=p_null, alternative='greater')  # testing if better than random
    print(f"{n} proteins: {x}/{k} correct (random = {p_null:.2f}) -> p = {result.pvalue:.4f}")


2 proteins: 15/37 correct (random = 0.50) -> p = 0.9061
3 proteins: 10/28 correct (random = 0.33) -> p = 0.4645
4 proteins: 3/3 correct (random = 0.25) -> p = 0.0156
5 proteins: 9/42 correct (random = 0.20) -> p = 0.4691
6 proteins: 1/5 correct (random = 0.17) -> p = 0.5981
8 proteins: 0/1 correct (random = 0.12) -> p = 1.0000


In [ ]:
num_phages

[37, 28, 3, 42, 5, 1]

In [ ]:
[int(x) for x in num_correct]

[32, 16, 1, 24, 4, 0]

In [ ]:
group_sizes

[2, 3, 4, 5, 6, 8]